# Install and Import Dependencies

In [2]:
pip install opencv-python numpy matplotlib mediapipe tensorflow scikit-learn tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np
import gc
import os
from matplotlib import pyplot as plt
import shutil
import threading
import mediapipe as mp
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tqdm import tqdm

# Load dataset

In [5]:
# defining a base directory to the dataset folder
main_path = './dataset/'
# opening the json file for read
with open(main_path + 'WLASL_v0.3.json', 'r') as json_file:
    data = json.load(json_file)

In [6]:
# checking to see if data has loaded properly 
len(data[0]['instances'])

40

# Create Labels and Features

In [7]:
# Open the file in read mode
with open(main_path+'missing.txt', 'r') as file:
    # Read all lines from the file and store them in a list
    missing_video_ids = [line.strip() for line in file.readlines()]

# Now, 'lines' contains an array of strings, where each element is a line from the file
print(missing_video_ids)

[]


In [8]:
del file, json_file
gc.collect()

7

In [9]:
gloss_list = [] 
video_name_list = []
start_frame = []
end_frame = []
category_list=[]
vid_dir = main_path + '/videos'
progress = 0
data_count = 0

# looping through each word / gesture 
for word in data:
    # get the all instances of each word for easy testing of code
    for i in range(len(word['instances'])):

        # if video is missing, skip it
        if any(word["instances"][i]["video_id"] in line for line in missing_video_ids):
            continue

        # get the video path 
        vid_name = os.path.join(vid_dir, f'{word["instances"][i]["video_id"]}.mp4')

        # get the start, end and the label 
        start = word['instances'][i]['frame_start']
        end = word['instances'][i]['frame_end']
        label = word['gloss']
        category=word['instances'][i]['split']

        # adding to the list
        video_name_list.append(vid_name)
        gloss_list.append(label)
        start_frame.append(start)
        end_frame.append(end)
        category_list.append(category)

        # updating progress after each loop
        progress += 1
        data_count +=1

print(data_count)

21083


In [10]:
unique_words_count = len(set(gloss_list))
print(f'number of unique words = {unique_words_count}\nnumber of videos = {len(video_name_list)}')
print(video_name_list)

number of unique words = 2000
number of videos = 21083
['./dataset//videos\\69241.mp4', './dataset//videos\\65225.mp4', './dataset//videos\\68011.mp4', './dataset//videos\\68208.mp4', './dataset//videos\\68012.mp4', './dataset//videos\\70212.mp4', './dataset//videos\\70266.mp4', './dataset//videos\\07085.mp4', './dataset//videos\\07086.mp4', './dataset//videos\\07087.mp4', './dataset//videos\\07069.mp4', './dataset//videos\\07088.mp4', './dataset//videos\\07089.mp4', './dataset//videos\\07090.mp4', './dataset//videos\\07091.mp4', './dataset//videos\\07092.mp4', './dataset//videos\\07093.mp4', './dataset//videos\\07068.mp4', './dataset//videos\\07094.mp4', './dataset//videos\\07095.mp4', './dataset//videos\\07096.mp4', './dataset//videos\\07097.mp4', './dataset//videos\\07070.mp4', './dataset//videos\\07098.mp4', './dataset//videos\\07099.mp4', './dataset//videos\\07071.mp4', './dataset//videos\\07072.mp4', './dataset//videos\\07073.mp4', './dataset//videos\\67424.mp4', './dataset//vide

In [11]:
del data, unique_words_count, video_name_list, word, progress
gc.collect()

14

# Collect Keypoint Values From Videos

### Mediapipe Implementation

In [12]:
# Create a Holistic object to detect pose, face, and hands keypoints
mp_holistic = mp.solutions.holistic

# Drawing utilities
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    if image is None:
        raise ValueError(f"Failed to load image: {image}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

### Centering The Landmarks

In [13]:
# def adjust_landmarks(arr,center):

#     # Reshape the array to have shape (n, 3)
#     arr_reshaped = arr.reshape(-1, 3)

#     # Repeat the center array to have shape (n, 3)
#     center_repeated = np.tile(center, (len(arr_reshaped), 1))

#     # Subtract the center array from the arr array
#     arr_adjusted = arr_reshaped - center_repeated

#     # Reshape arr_adjusted back to shape (n*3,)
#     arr_adjusted = arr_adjusted.reshape(-1)
#     return(arr_adjusted)

In [14]:
def adjust_3d_landmarks(arr, center):
    # the array reshaped to have shape (n*m, 3)
    arr_reshaped = arr.reshape(-1, 3)

    # the center array repeated to have shape (n*m, 3)
    center_repeated = np.tile(center, (len(arr_reshaped), 1))

    # the center array subtracted from the arr array
    arr_adjusted = arr_reshaped - center_repeated

    # arr_adjusted reshaped back to shape (n, m, 3)
    arr_adjusted = arr_adjusted.reshape(arr.shape)

    return arr_adjusted

### Extracting Keypoints From A Frame

In [15]:
# def extract_keypoints(results):

#     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
#     lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
#     rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
#     nose=pose[:3]
#     lh_wrist=lh[:3]
#     rh_wrist=rh[:3]
#     face_landmark=face[:3]
#     pose_adjusted = adjust_landmarks(pose,nose)
#     lh_adjusted = adjust_landmarks(lh,lh_wrist)
#     rh_adjusted = adjust_landmarks(rh,rh_wrist)
#     face_adjusted = adjust_landmarks(face, face_landmark)
#     return np.concatenate([pose_adjusted, face_adjusted, lh_adjusted, rh_adjusted])

In [16]:
filtered_face = [0, 4, 7, 8, 10, 13, 14, 17, 21, 33, 37, 39, 40, 46, 52, 53, 54, 55, 58,
                 61, 63, 65, 66, 67, 70, 78, 80, 81, 82, 84, 87, 88, 91, 93, 95, 103, 105,
                 107, 109, 127, 132, 133, 136, 144, 145, 146, 148, 149, 150, 152, 153, 154,
                 155, 157, 158, 159, 160, 161, 162, 163, 172, 173, 176, 178, 181, 185, 191,
                 234, 246, 249, 251, 263, 267, 269, 270, 276, 282, 283, 284, 285, 288, 291,
                 293, 295, 296, 297, 300, 308, 310, 311, 312, 314, 317, 318, 321, 323, 324,
                 332, 334, 336, 338, 356, 361, 362, 365, 373, 374, 375, 377, 378, 379, 380,
                 381, 382, 384, 385, 386, 387, 388, 389, 390, 397, 398, 400, 402, 405, 409,
                 415, 454, 466, 468, 473]

filtered_pose = [11, 12, 13, 14, 15, 16]

FACE_NUM = len(filtered_face)
POSE_NUM = len(filtered_pose)
HAND_NUM = 21 # per hand

In [17]:
hands = mp.solutions.hands.Hands()
pose = mp.solutions.pose.Pose()
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

def get_frame_landmarks(frame):
    
    all_landmarks = np.zeros((HAND_NUM * 2 + POSE_NUM + FACE_NUM, 3))
    
    def get_hands(frame):
        results_hands = hands.process(frame)
        if results_hands.multi_hand_landmarks:
            # print("HAND:")
            # print(results_hands.multi_hand_landmarks)
            for i, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
                if results_hands.multi_handedness[i].classification[0].index == 0:
                    hands_landmarks = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])
                    hands_center = hands_landmarks[0, :3]
                    hands_adjusted = adjust_3d_landmarks(hands_landmarks, hands_center)
                    all_landmarks[:HAND_NUM, :] = hands_adjusted
                else:
                    hands_landmarks = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])
                    hands_center = hands_landmarks[0, :3]
                    hands_adjusted = adjust_3d_landmarks(hands_landmarks, hands_center)
                    all_landmarks[HAND_NUM:HAND_NUM * 2, :] = hands_adjusted

            print("HAND:")
            print(hands_adjusted)

    def get_pose(frame):
        results_pose = pose.process(frame)
        if results_pose.pose_landmarks:
            # print("POSE: ")
            # print(results_pose.pose_landmarks)
            poses_landmarks = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_pose.pose_landmarks.landmark])[filtered_pose]
            pose_center = poses_landmarks[0, :3]
            pose_adjusted = adjust_3d_landmarks(poses_landmarks, pose_center)
            all_landmarks[HAND_NUM * 2:HAND_NUM * 2 + POSE_NUM, :] = pose_adjusted
        
    def get_face(frame):
        results_face = face_mesh.process(frame)
        if results_face.multi_face_landmarks:            
            # print("FACE: ")
            # print(results_face.multi_face_landmarks)
            face = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_face.multi_face_landmarks[0].landmark])[filtered_face]
            face_center = face[0, :3]
            face_adjusted = adjust_3d_landmarks(face, face_center)
            all_landmarks[HAND_NUM * 2 + POSE_NUM:, :] = face_adjusted
        
    threads = []
    threads.append(threading.Thread(target=get_hands, args=(frame,)))
    threads.append(threading.Thread(target=get_pose, args=(frame,)))
    threads.append(threading.Thread(target=get_face, args=(frame,)))
    
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    return all_landmarks

### Extracting Keypoints From A Video

In [18]:
# # processes video frame by frame to get landmarks for pose, hand and face using mediapipe 
# def extract_landmarks_from_video(video_path, start_frame=1, end_frame=-1):

#     # initiliaze video capture object at the video path 
#     cap = cv2.VideoCapture(video_path)

#     # checks if the frame are within range 
#     if end_frame < 0:
#         end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     if start_frame < 1:
#         start_frame = 1
    
#     landmarks_data = []
    
#     # setting up instances of the mediapipe modules
#     with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        
#         # loops through the video frames 
#         while cap.isOpened() and start_frame <= end_frame:
#             # reads frame 
#             ret, frame = cap.read()
#             if not ret:
#                 break

#             # convert to rgb format
#             frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             # Make detections
#             frame_rgb, results = mediapipe_detection(frame_rgb, holistic)
#             # Get the results
#             keypoints = extract_keypoints(results)
#             landmarks_data.append(keypoints)
#             start_frame += 1

#     # release video after processing is complete 
#     cap.release()
#     return landmarks_data # contains landmarks for each frame

In [19]:
def get_video_landmarks(video_path, start_frame=1, end_frame=-1):
    cap = cv2.VideoCapture(video_path)
    
    # if the starting is 0
    if start_frame <= 1:
        start_frame = 1
        
    # if the video is precropped
    elif start_frame > int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
        start_frame = 1
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
    # if the final frame was not given (-1)    
    if end_frame < 0: 
        end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    num_landmarks = HAND_NUM * 2 + POSE_NUM + FACE_NUM
    all_frame_landmarks = np.zeros((end_frame - start_frame + 1, num_landmarks, 3))
    frame_index = 1
    
    while cap.isOpened() and frame_index <= end_frame:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_index >= start_frame:
            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_landmarks = get_frame_landmarks(frame)
            all_frame_landmarks[frame_index - start_frame] = frame_landmarks

        frame_index += 1

    cap.release()
    hands.reset()
    pose.reset()
    face_mesh.reset()
    return all_frame_landmarks

In [20]:
def process_video(landmarks,frame_count):
    # stacks the landmarks for all frames and creates a list of the results
    stacked_arrays = [(landmarks[i]) for i in range(frame_count)]
    return stacked_arrays

# Collecting And Saving Keypoints

### Setup Folders for Collection Of .npy files

In [21]:
# store the keypoints in a folder
numpy_dir = main_path + 'new_preprocessed-data'
if not os.path.exists(numpy_dir):
    os.mkdir(numpy_dir)

numpy_dir_1 = numpy_dir + '/train'
numpy_dir_2 = numpy_dir + '/val'
numpy_dir_3 = numpy_dir + '/test'
if not os.path.exists(numpy_dir_1):
    os.mkdir(numpy_dir_1)
if not os.path.exists(numpy_dir_2):
    os.mkdir(numpy_dir_2)
if not os.path.exists(numpy_dir_3):
    os.mkdir(numpy_dir_3)

### Encoding And Saving Data

In [22]:
# data_count is the number of dataset 

for i in tqdm(range(data_count), ncols=100):
    # specify the path for the test npy to be saved  
    npy_path = f'{main_path}new_preprocessed-data/{category_list[i]}/{i}.npy'
    # npy_path = f'{main_path}new_preprocessed-data/new/{i}.npy'

    # Check if the file already exists
    if os.path.exists(npy_path):
        print(f"File {npy_path} already exists..")
        continue
    
    # getting the video information 
    input_path = video_name_list[i]
    start = start_frame[i]
    end = end_frame[i]

    
    # get the landmarks for the specified frame range  
    npyArray = get_video_landmarks(input_path, start, end)
    # print(npyArray)
  
    # # save the file 
    np.save(npy_path,npyArray)
    print(f"{category_list[i]}/{i}.npy has been saved")

  0%|                                                             | 6/21083 [00:00<09:51, 35.61it/s]

File ./dataset/new_preprocessed-data/train/0.npy already exists..
File ./dataset/new_preprocessed-data/train/1.npy already exists..
File ./dataset/new_preprocessed-data/train/2.npy already exists..
File ./dataset/new_preprocessed-data/train/3.npy already exists..
File ./dataset/new_preprocessed-data/train/4.npy already exists..
File ./dataset/new_preprocessed-data/val/5.npy already exists..
File ./dataset/new_preprocessed-data/train/6.npy already exists..
File ./dataset/new_preprocessed-data/train/7.npy already exists..
File ./dataset/new_preprocessed-data/train/8.npy already exists..
File ./dataset/new_preprocessed-data/train/9.npy already exists..
File ./dataset/new_preprocessed-data/train/10.npy already exists..
File ./dataset/new_preprocessed-data/train/11.npy already exists..
File ./dataset/new_preprocessed-data/train/12.npy already exists..
File ./dataset/new_preprocessed-data/train/13.npy already exists..
File ./dataset/new_preprocessed-data/train/14.npy already exists..


  0%|▎                                                          | 91/21083 [00:00<01:05, 319.66it/s]

File ./dataset/new_preprocessed-data/test/15.npy already exists..
File ./dataset/new_preprocessed-data/test/16.npy already exists..
File ./dataset/new_preprocessed-data/train/17.npy already exists..
File ./dataset/new_preprocessed-data/val/18.npy already exists..
File ./dataset/new_preprocessed-data/test/19.npy already exists..
File ./dataset/new_preprocessed-data/val/20.npy already exists..
File ./dataset/new_preprocessed-data/train/21.npy already exists..
File ./dataset/new_preprocessed-data/train/22.npy already exists..
File ./dataset/new_preprocessed-data/val/23.npy already exists..
File ./dataset/new_preprocessed-data/val/24.npy already exists..
File ./dataset/new_preprocessed-data/train/25.npy already exists..
File ./dataset/new_preprocessed-data/test/26.npy already exists..
File ./dataset/new_preprocessed-data/train/27.npy already exists..
File ./dataset/new_preprocessed-data/train/28.npy already exists..
File ./dataset/new_preprocessed-data/train/29.npy already exists..
File ./

  1%|▊                                                         | 314/21083 [00:00<00:27, 761.04it/s]

...

 96%|████████████████████████████████████████████████████▊  | 20221/21083 [00:08<00:00, 3049.47it/s]

File ./dataset/new_preprocessed-data/train/19513.npy already exists..
File ./dataset/new_preprocessed-data/train/19514.npy already exists..
File ./dataset/new_preprocessed-data/test/19515.npy already exists..
File ./dataset/new_preprocessed-data/val/19516.npy already exists..
File ./dataset/new_preprocessed-data/train/19517.npy already exists..
File ./dataset/new_preprocessed-data/train/19518.npy already exists..
File ./dataset/new_preprocessed-data/train/19519.npy already exists..
File ./dataset/new_preprocessed-data/train/19520.npy already exists..
File ./dataset/new_preprocessed-data/train/19521.npy already exists..
File ./dataset/new_preprocessed-data/test/19522.npy already exists..
File ./dataset/new_preprocessed-data/val/19523.npy already exists..
File ./dataset/new_preprocessed-data/train/19524.npy already exists..
File ./dataset/new_preprocessed-data/train/19525.npy already exists..
File ./dataset/new_preprocessed-data/train/19526.npy already exists..
File ./dataset/new_preproc

 99%|██████████████████████████████████████████████████████▋| 20957/21083 [00:08<00:00, 3333.50it/s]

File ./dataset/new_preprocessed-data/train/20416.npy already exists..
File ./dataset/new_preprocessed-data/train/20417.npy already exists..
File ./dataset/new_preprocessed-data/test/20418.npy already exists..
File ./dataset/new_preprocessed-data/val/20419.npy already exists..
File ./dataset/new_preprocessed-data/train/20420.npy already exists..
File ./dataset/new_preprocessed-data/train/20421.npy already exists..
File ./dataset/new_preprocessed-data/train/20422.npy already exists..
File ./dataset/new_preprocessed-data/train/20423.npy already exists..
File ./dataset/new_preprocessed-data/train/20424.npy already exists..
File ./dataset/new_preprocessed-data/test/20425.npy already exists..
File ./dataset/new_preprocessed-data/val/20426.npy already exists..
File ./dataset/new_preprocessed-data/train/20427.npy already exists..
File ./dataset/new_preprocessed-data/train/20428.npy already exists..
File ./dataset/new_preprocessed-data/train/20429.npy already exists..
File ./dataset/new_preproc

100%|███████████████████████████████████████████████████████| 21083/21083 [00:08<00:00, 2475.24it/s]


In [23]:
train_landmarks_dict = {}
test_landmarks_dict = {}
val_landmarks_dict = {}

train_npy_dir = main_path + 'new_preprocessed-data/train'
test_npy_dir = main_path + 'new_preprocessed-data/test'
val_npy_dir = main_path + 'new_preprocessed-data/val'

In [24]:
def generate_npz(npy_dir, landmarks_dict, npz_path):
    for filename in os.listdir(npy_dir):
        if filename.endswith('.npy'):
            key = filename.split('.')[0]
            landmarks = np.load(os.path.join(npy_dir, filename), allow_pickle=True)
            # print(landmarks)
            landmarks_dict[key] = landmarks
            print(f'{npy_dir}/{filename} added')

    np.savez_compressed(npz_path, **landmarks_dict)
    print(f"{npy_dir} saved successfully")

In [13]:
npz_landmarks_dir = main_path + 'landmarks/'
if not os.path.exists(npz_landmarks_dir):
    os.mkdir(npz_landmarks_dir)

In [26]:
npz_train_path = main_path + 'landmarks/landmarks_train.npz'
npz_test_path = main_path + 'landmarks/landmarks_test.npz'
npz_val_path = main_path + 'landmarks/landmarks_val.npz'

# generate_npz(train_npy_dir, train_landmarks_dict, npz_train_path)

if not os.path.exists(npz_train_path):
    generate_npz(train_npy_dir, train_landmarks_dict, npz_train_path)
if not os.path.exists(npz_test_path):
    generate_npz(test_npy_dir, test_landmarks_dict, npz_test_path)
if not os.path.exists(npz_val_path):
    generate_npz(val_npy_dir, val_landmarks_dict, npz_val_path)

In [27]:
del train_landmarks_dict, test_landmarks_dict, val_landmarks_dict
gc.collect()

9

In [28]:
# train_landmarks_dict = {}
# train_npy_dir = main_path + 'new_preprocessed-data/new'
# npz_train_path = main_path + 'landmarks_new.npz'

# generate_npz(train_npy_dir, train_landmarks_dict, npz_train_path)

# Data Loading

In [14]:
landmark_train_dict = np.load(npz_landmarks_dir + 'landmarks_train.npz', allow_pickle=True)
landmark_test_dict = np.load(npz_landmarks_dir + 'landmarks_test.npz', allow_pickle=True)
landmark_val_dict = np.load(npz_landmarks_dir + 'landmarks_val.npz', allow_pickle=True)

In [15]:
def load_data(landmarks_dict):
    
    X = [landmarks_dict[k] for k in landmarks_dict.keys()]
    Y = [gloss_list[int(k)] for k in landmarks_dict.keys()]
    
    return X, Y

In [16]:
X_train, Y_train = load_data(landmark_train_dict)
X_val, Y_val = load_data(landmark_val_dict)
X_test, Y_test = load_data(landmark_test_dict)

In [32]:
len(X_train), len(X_val), len(X_test)

(14289, 3916, 2878)

In [33]:
len(np.unique(Y_train)), len(np.unique(Y_val)), len(np.unique(Y_test))

(2000, 2000, 1999)

In [17]:
# free space
del landmark_train_dict, landmark_test_dict, landmark_val_dict
gc.collect()

261

In [35]:
# landmark_train_dict = np.load(main_path + 'landmarks_new.npz', allow_pickle=True)
# X_train, Y_train = load_data(landmark_train_dict)

# # print(X_train)
# # print(len(X_train))
# print(X_train)

# Data Augmentation

### Rotation Augmentation

In [18]:
def rotate(data, rotation_matrix):
    frames, landmarks, _ = data.shape
    center = np.array([0.5, 0.5, 0])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data = data.reshape(-1, 3)
    data[non_zero] -= center
    data[non_zero] = np.dot(data[non_zero], rotation_matrix.T)
    data[non_zero] += center
    data = data.reshape(frames, landmarks, 3)
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data


def rotate_z(data):
    angle = np.random.choice([np.random.uniform(-15, -5),
                              np.random.uniform(5, 15)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])
    return rotate(data, rotation_matrix)


def rotate_y(data):
    angle = np.random.choice([np.random.uniform(-15, -5),
                              np.random.uniform(5, 15)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)


def rotate_x(data):
    angle = np.random.choice([np.random.uniform(-15, -5),
                              np.random.uniform(5, 15)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)

### Other Augmentations

In [19]:
def zoom(data):
    factor = np.random.uniform(0.8, 1.2)
    center = np.array([0.5, 0.5])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], :2] = (
        (data[non_zero[:, 0], non_zero[:, 1], :2] - center) * factor + center
    )
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data


def shift(data):
    x_shift = np.random.uniform(-0.1, 0.1)
    y_shift = np.random.uniform(-0.1, 0.1)
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], 0] += x_shift
    data[non_zero[:, 0], non_zero[:, 1], 1] += y_shift
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data


def mask(data):
    frames, landmarks, _ = data.shape
    num_hands = int(0.3 * 42)
    num_rest = int(0.6 * (landmarks - 42))

    mask = np.zeros(landmarks, dtype=bool)
    indices = np.concatenate([
        np.random.choice(42, num_hands, replace=False),
        np.random.choice(landmarks - 42, num_rest, replace=False) + 42
    ])
    mask[indices] = True
    data[:, mask] = 0
    return data


def hflip(data):
    data[:, :, 0] = 1 - data[:, :, 0]
    return data


def speedup(data):
    return data[::2]

### Applying the Augmentations to the Data

In [20]:
def apply_augmentations(data):
    aug_functions = [rotate_x, rotate_y, rotate_z,
                     zoom, shift, mask, hflip, speedup]
    np.random.shuffle(aug_functions)
    counter = 0
    for fun in aug_functions:
        if np.random.rand() < 0.5:
            data = fun(data)
            counter += 1

    if counter == 0:
        data = apply_augmentations(data)

    return data

In [21]:
def augment(X, Y, num=None):
    X_aug = X.copy()
    Y_aug = Y.copy()

    if num is None:
        for i in tqdm(range(len(Y)), ncols=100):
            num_aug = np.random.choice([1, 2, 3])
            for n in range(num_aug):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])
    elif num > 0:
        for i in tqdm(range(len(Y)), ncols=100):
            for n in range(num):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])

    return X_aug, Y_aug

In [22]:
X_train, Y_train = augment(X_train, Y_train, 1)

  0%|                                                                     | 0/14289 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████| 14289/14289 [03:16<00:00, 72.88it/s]


In [41]:
len(X_train), len(Y_train)

(28578, 28578)

In [43]:
len(X_train), len(X_train[339]), len(X_train[12][12]), len(X_train[0][0][0])

(28578, 55, 180, 3)

# Data Preprocessing

In [24]:
def padding(X, Y, length=None, pad=0):
    if length is None:
        length = max(len(x) for x in X)
    
    X_padded = []
    for x in X:
        if len(x) > length:
            X_padded.append(x[:length]) #truncate
        else:
            pad_length = length - len(x)
            X_padded.append(np.pad(
                x, ((0, pad_length), (0, 0), (0, 0)),
                mode='constant', constant_values=pad
            ))
            
    X_padded = np.array(X_padded)
    return X_padded, Y

In [26]:
gc.collect()
length = 100
X_train, Y_train = padding(X_train, Y_train, length=length, pad=-80)
X_val, Y_val = padding(X_val, Y_val, length=length, pad=-80)
X_test, Y_test = padding(X_test, Y_test, length=length, pad=-80)

In [27]:
X_train.shape, X_val.shape, X_test.shape

((28578, 60, 180, 3), (3916, 60, 180, 3), (2878, 60, 180, 3))

# Build and Train LSTM Neural Network

### Label Encoding

In [28]:
labels = np.unique(Y_train)
unique_labels = {word: idx for idx, word in enumerate(labels)}
print(len(unique_labels))

2000


In [29]:
Y_train = np.array([unique_labels.get(label, label) for label in Y_train])
Y_val = np.array([unique_labels.get(label, label) for label in Y_val])
Y_test = np.array([unique_labels.get(label, label) for label in Y_test])

In [30]:
Y_train = Y_train.astype(np.int32)
Y_val = Y_val.astype(np.int32)
Y_test = Y_test.astype(np.int32)

In [32]:
Y_train, Y_val, Y_test

(array([210, 210, 210, ..., 693, 693, 693]),
 array([693, 723, 994, ..., 994, 689, 693]),
 array([723, 723, 730, ..., 689, 693, 693]))

In [33]:
len(X_train), Y_train.shape, Y_val.shape, Y_test.shape

(28578, (28578,), (3916,), (2878,))

In [34]:
# Iterate through the list and print the data type of each array
for i, arr in enumerate(Y_train):
    if np.issubdtype(arr.dtype, np.float_):
        print(f"Array {i + 1} has a float data type.")

In [35]:
print(f"X_train_aug data type: {X_train.dtype}")
print(f"Y_train_aug data type: {Y_train.dtype}")
print(f"Y_val data type: {Y_val.dtype}")

X_train_aug data type: float64
Y_train_aug data type: int32
Y_val data type: int32


In [36]:
for i in Y_train:
    if i.dtype != np.int32:
        print(i.dtype)

In [37]:
# del arr, element_types, end, json_file, label, pose, progress, word
# del category, category_list, data_count, end_frame, face_mesh, FACE_NUM, filtered_face, filtered_pose, HAND_NUM, hands, json_file, label, labels, missing_video_ids, pose, start, start_frame, word
gc.collect()

485

### Defining Model

In [38]:
tf.keras.backend.clear_session()
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
physical_devices

[]

In [45]:
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=-100, input_shape=(60, 180, 3)),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.3),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.3),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(300, activation='linear')
])

In [46]:
checkpoint_filepath = main_path + 'model-checkpoints/'
os.makedirs(checkpoint_filepath, exist_ok=True)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_filepath, 'model_{epoch:02d}.h5'),
    save_weights_only=True,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=50,
    restore_best_weights=True
)

lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9
)

model.compile(
    loss=tf.keras.losses.cosine_similarity,
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler),
    metrics=['accuracy']
)

### Training And Validation of Model

In [47]:
gc.collect()
hist = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=2,
    batch_size=128,
    callbacks=[model_checkpoint, early_stopping]
)

Epoch 1/2


TypeError: in user code:

    File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1151

### Testing of Model

In [67]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 80, 180, 3)        0         
                                                                 
 flatten (Flatten)           (None, 43200)             0         
                                                                 
 dense (Dense)               (None, 512)               22118912  
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                        

# Keypoints using MP Holistic

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()